# Explore the OpenBreweryDB API 

In [ ]:
https://sandbox-api.brewerydb.com/v2/
Sandbox Key:c13c821fb670e2fb8cf9a31b4d335d96

In [ ]:
brewery_url = "https://sandbox-api.brewerydb.com/v2/"
brewery_data = requests.get(brewery_url).json()
pprint(brewery_data)

In [ ]:
# brewery_url = "https://api.openbrewerydb.org/breweries?per_page=25"
# brewery_data = requests.get(brewery_url).json()
# pprint(brewery_data)

In [ ]:
# brewery_df = pd.DataFrame(brewery_data)
# brewery_df

# Exploring the CSV format 

In [ ]:
# Read in the CSV using pandas and save into dataframe 
brewery_df = pd.read_csv("https://raw.githubusercontent.com/openbrewerydb/openbrewerydb/master/breweries.csv")
brewery_df

In [ ]:
# Overview of brewery dataframe 
brewery_df.info()

In [ ]:
# Identify and count null values in each column in df 
nan_count = brewery_df.isnull().sum()
nan_count[nan_count > 0].sort_values(ascending = False)

In [ ]:
# Drop the columns that contain only null values (notes and tags)
brewery_df = brewery_df.drop(['notes', 'tags'], axis = 1)

### Dropping unecessary columns 
created_at, updated_at

In [ ]:
brewery_df.info()

In [ ]:
brewery_df = brewery_df.drop(['created_at', 'updated_at'], axis = 1)

In [ ]:
brewery_df["name"].value_counts()

### Standardising the zip code column 


In [ ]:
# Check the min and mac length of values in the postal Code column 
postal_check = brewery_df['postal_code'].apply(len)
print(f"Max zip_code length is {postal_check.max()}.")
print(f"Min zip_code length is {postal_check.min()}.")

In [ ]:
# Check the rows where the postal code length is 3 and drop it 
brewery_df[brewery_df.postal_code.str.len() == postal_check.min()]
brewery_df = brewery_df.loc[~(brewery_df['postal_code'] == "TBD")]
brewery_df.head()

In [ ]:
# Get rid of the rows that have 4 digit postal codes and null lat and long values
brewery_df = brewery_df.loc[~((brewery_df['postal_code'].str.len() == 4) &
                             (brewery_df['longitude'].isnull())),:]

In [ ]:
brewery_df.info()

In [ ]:
brewery_df[brewery_df.postal_code.str.len() == 5]

In [ ]:
# Clean the postal codes and retain only the first 5 characters 
brewery_df['postal_code'] = brewery_df['postal_code'].apply(lambda x: x.split("-")[0].strip())

In [ ]:
brewery_df[brewery_df.postal_code.str.len() == 5]

In [ ]:
# Verify that all postal codes have 5 characters only 
postal_check = brewery_df['postal_code'].apply(len)
print(f"Max zip_code length is {postal_check.max()}.")
print(f"Min zip_code length is {postal_check.min()}.")

# Google Maps API to obtain lat and lon lists 

In [1]:
# Import dependencies 
import pandas as pd 
import numpy as np
import requests 
import json
from pprint import pprint

In [2]:
df = pd.read_csv("../../Data/beer.gz", encoding="ISO-8859-1")
df.head(20)

,Unnamed: 0,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,0,10325,Vecchio Birraio,2009-02-16 20:57:03,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,1,10325,Vecchio Birraio,2009-03-01 13:44:57,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,2,10325,Vecchio Birraio,2009-03-01 14:10:04,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,3,10325,Vecchio Birraio,2009-02-15 19:12:25,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,4,1075,Caldera Brewing Company,2010-12-30 18:53:26,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883
5,5,1075,Caldera Brewing Company,2012-01-02 17:17:39,3.0,3.5,3.5,oline73,Herbed / Spiced Beer,3.0,3.5,Caldera Ginger Beer,4.7,52159
6,6,1075,Caldera Brewing Company,2011-10-19 02:25:15,3.5,3.5,3.5,Reidrover,Herbed / Spiced Beer,4.0,4.0,Caldera Ginger Beer,4.7,52159
7,7,1075,Caldera Brewing Company,2011-05-24 22:26:58,3.0,2.5,3.5,alpinebryant,Herbed / Spiced Beer,2.0,3.5,Caldera Ginger Beer,4.7,52159
8,8,1075,Caldera Brewing Company,2010-11-22 19:35:03,4.0,3.0,3.5,LordAdmNelson,Herbed / Spiced Beer,3.5,4.0,Caldera Ginger Beer,4.7,52159
9,9,1075,Caldera Brewing Company,2010-09-28 00:15:24,4.5,3.5,5.0,augustgarage,Herbed / Spiced Beer,4.0,4.0,Caldera Ginger Beer,4.7,52159


In [3]:
brewery = df['brewery_name']
brewery.drop_duplicates(keep = 'first', inplace = True) 
brewery_names = brewery.to_numpy()

In [4]:
brewery_names

array(['Vecchio Birraio', 'Caldera Brewing Company',
       'Amstel Brouwerij B. V.', ..., 'Wissey Valley Brewery',
       'Outback Brewery Pty Ltd', 'Georg Meinel Bierbrauerei KG'],
      dtype=object)

In [ ]:
https://maps.googleapis.com/maps/api/place/autocomplete/json?input=Vict&types=(cities)&language=pt_BR&key=YOUR_API_KEY

In [8]:
# obtaining the location of each developer's office using google maps
lat = []
lng = []

params={
    "inputtype": "textquery",
    "types": "establishment",
    "key": g_key,
}

for name in brewery_names:
    params["inputtype"] = name
    url = "https://maps.googleapis.com/maps/api/place/autocomplete/json?parameters"
    response = requests.get(url, params = params).json()
    
    try:
        place_id = response["predictions"][0]["place_id"]
        url2 = "https://maps.googleapis.com/maps/api/geocode/json?place_id="+place_id+"&key="+g_key
        response2 = requests.get(url2).json()
        lat.append(response2["results"][0]["geometry"]["location"]["lat"])
        lng.append(response2["results"][0]["geometry"]["location"]["lng"])
        
    except:
        lat.append("missing")
        lng.append("missing")

In [10]:
lng

['missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'missing',
 'mi

In [ ]:
# obtaining the location of each developer's office using google maps
lat = []
lng = []

params={
    "inputtype":"textquery",
    "key":g_key,
}

for i in dev_gamecount.index:
    params["input"]=i
    url="https://maps.googleapis.com/maps/api/place/autocomplete/json?parameters"
    response=requests.get(url, params=params).json()
    try:
        place_id=response["predictions"][0]["place_id"]
        url2="https://maps.googleapis.com/maps/api/geocode/json?place_id="+place_id+"&key="+g_key
        response2=requests.get(url2).json()
        lat.append(response2["results"][0]["geometry"]["location"]["lat"])
        lng.append(response2["results"][0]["geometry"]["location"]["lng"])
    except:
        lat.append("missing")
        lng.append("missing")